## Required Libraries (or) Modules

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split

## Neural Network model

In [2]:
class NeuralNetwork:
    def __init__(self, input_size, hidden_layers, output_size):
        self.input_size = input_size
        self.hidden_layers = hidden_layers
        self.output_size = output_size
        self.weights = []
        self.biases = []
        self.weights.append(np.random.uniform(size=(self.input_size,self.hidden_layers[0])))
        self.biases.append(np.random.uniform(size=(1,self.hidden_layers[0])))
        for i in range(1,len(hidden_layers)):
            self.weights.append(np.random.uniform(size=(self.hidden_layers[i-1],self.hidden_layers[i])))
            self.biases.append(np.random.uniform(size=(1,self.hidden_layers[i])))
        self.weights.append(np.random.uniform(size=(self.hidden_layers[-1],self.output_size)))
        self.biases.append(np.random.uniform(size=(1,self.output_size)))
    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))
    
    def mean_absolute_error(self, y_true, y_pred):
        return np.mean(np.abs(y_true - y_pred))
    
    def accuracy(self, y_true, y_pred):
        true_classes = np.argmax(y_true, axis=1)
        predicted_classes = np.argmax(y_pred, axis=1)
        return np.mean(true_classes == predicted_classes)

    def feedforward(self, X):
        self.activations=[X]
        for i in range(len(self.hidden_layers)):
            hid_input = np.dot(self.activations[i], self.weights[i]) + self.biases[i]
            hid_output = self.sigmoid(hid_input)
            self.activations.append(hid_output)
        final_input= np.dot(self.activations[-1], self.weights[-1]) + self.biases[-1]
        final_output = self.sigmoid(final_input)
        self.activations.append(final_output)
        return final_output

    def backward(self, X, y, learning_rate):
        errors = [y - self.activations[-1]]
        final_errors = [self.activations[-1]*(1-self.activations[-1])*errors[0]]
        for i in range(len(self.hidden_layers), 0, -1):
            error = final_errors[-1].dot(self.weights[i].T)
            hid_errors = self.activations[i]*(1-self.activations[i])*error
            errors.append(error)
            final_errors.append(hid_errors)
        final_errors.reverse()
        for i in range(len(self.weights)):
            self.weights[i] += self.activations[i].T.dot(final_errors[i]) * learning_rate
            self.biases[i] += np.sum(final_errors[i]) * learning_rate

    def train(self, X, y, epochs, learning_rate):
        for epoch in range(epochs):
            output = self.feedforward(X)
            self.backward(X, y, learning_rate)
            
            if epoch % 1000 == 0:
                loss = -np.mean(np.sum(y * np.log(output + 1e-9), axis=1))
                print(f"Epoch {epoch}, Loss: {loss}")
            

## Upload Your CSV File

In [3]:
    try:
        file_path = input("Enter the path to your CSV file: ")
        dataset = pd.read_csv(file_path)
    except FileNotFoundError:
        print("Error: File not found. Please check the file path.")

Enter the path to your CSV file: iris.csv


## Seperation of Features and Labels

In [5]:
    X = dataset.iloc[:, :-1]
    y = dataset.iloc[:, -1]

## Data Preprocessing: Encoding and Scaling

In [6]:
    label_encoders = {}
    scaler = StandardScaler()
    for column in X.columns:
        if X[column].dtype == 'object': 
            le = LabelEncoder()
            X[column] = le.fit_transform(X[column])  
            label_encoders[column] = le  
    X = scaler.fit_transform(X.values.astype(float))

    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y).reshape(-1, 1)
    onehot_encoder = OneHotEncoder(sparse_output=False)
    y_encoded = onehot_encoder.fit_transform(y_encoded)

## Splitting the dataset into training and testing data

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.3, random_state=42)

## Inputs for the requirements for implemenation of Neural Network model

In [15]:

    output_size = y_encoded.shape[1]
    input_size = X_train.shape[1]
    
    num_hidden_layers = int(input("Enter the number of hidden layers: "))
    hidden_layers = []
    for i in range(num_hidden_layers):
        size = int(input(f"Enter the size of hidden layer {i + 1}: "))
        hidden_layers.append(size)
    
    epochs = int(input("Enter the number of epochs: "))
    learning_rate = float(input("Enter the learning rate: "))
            

Enter the number of hidden layers: 1
Enter the size of hidden layer 1: 3
Enter the number of epochs: 10000
Enter the learning rate: 0.1


## Neural Network Setup and Training

In [16]:

    nn = NeuralNetwork(input_size=input_size, hidden_layers=hidden_layers, output_size=output_size)
    nn.train(X_train, y_train, epochs=epochs, learning_rate=learning_rate)
            

Epoch 0, Loss: 0.316967633313287
Epoch 1000, Loss: 0.07167950643527442
Epoch 2000, Loss: 0.06418168674769331
Epoch 3000, Loss: 0.0620889800792365
Epoch 4000, Loss: 0.062295416149016354
Epoch 5000, Loss: 0.0629569196445519
Epoch 6000, Loss: 0.06596093019981397
Epoch 7000, Loss: 0.06692710520127519
Epoch 8000, Loss: 0.06802828136314681
Epoch 9000, Loss: 0.06903374148742368


## Predicting Labels for testing data

In [17]:
    output = nn.feedforward(X_test)
    predictions = np.argmax(output, axis=1)
    true_labels = np.argmax(y_test, axis=1)

## Model Evaluation

In [18]:
    print("\nPredictions after training:")
    print(predictions)
    print("\nTrue Labels:")
    print(true_labels)
    test_predictions = nn.feedforward(X_test)
    test_mae = nn.mean_absolute_error(y_test, test_predictions)
    print(f"Test Mean Absolute Error: {test_mae}")
    test_accuracy = nn.accuracy(y_test, test_predictions)
    print(f"Test Accuracy: {test_accuracy * 100:.2f}%")


Predictions after training:
[1 0 2 1 1 0 1 2 1 1 2 0 0 0 0 1 2 1 1 2 0 2 0 2 2 2 2 2 0 0 0 0 1 0 0 2 1
 0 0 0 2 2 1 0 0]

True Labels:
[1 0 2 1 1 0 1 2 1 1 2 0 0 0 0 1 2 1 1 2 0 2 0 2 2 2 2 2 0 0 0 0 1 0 0 2 1
 0 0 0 2 1 1 0 0]
Test Mean Absolute Error: 0.02499339224677966
Test Accuracy: 97.78%
